In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_tracking/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)


In [ ]:
sessionA = 'classinsight-cmu_05748A_ghc_4101_201902051630'
sessionB = 'classinsight-cmu_05748A_ghc_4101_201902141630'
course = '05748A'


In [ ]:
sessionA_frame_dir = f'{base_dir}/{course}/{sessionA}-front'
sessionB_frame_dir = f'{base_dir}/{course}/{sessionB}-front'


## Get embedding and gaze information for all frames for all sessions (Run if needed, commented out for now)?


## Get frame file data for all sessions


In [ ]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    # if os.path.exists(course_cache_file):
    #     frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
    #     continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
        
frame_file_data.keys()


In [ ]:
# writing a generic loop to get embedding info from all courses in frame file data
emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info'
os.makedirs(emb_analysis_session_data,exist_ok=True)

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        session_emb_cache_file = f"{emb_analysis_session_data}/{session_id}.pb"
        try:
            if not os.path.exists(session_emb_cache_file):
                session_dir = frame_file_data[course][session_id]['dir_location']
                frame_ids = frame_file_data[course][session_id]['frame_ids']
                session_emb_info = {}
                for frame_id in frame_ids:
                    frame_number, frame_data = pickle.load(open(f'{session_dir}/{frame_id}.pb','rb'))
                    frame_emb_info = {int(person_info['track_id']):{
                        'bbox': person_info['bbox'] if 'bbox' in person_info else None,
                        'rvec': person_info['rvec'] if 'rvec' in person_info else None,
                        'gaze_2d':person_info['gaze_2d'] if 'gaze_2d' in person_info else None,
                        'face_embedding': person_info['face_embedding'] if 'face_embedding' in person_info else None,
                    } for person_info in frame_data}
                    session_emb_info[frame_id] = frame_emb_info
                pickle.dump(session_emb_info, open(session_emb_cache_file,'wb'))
                print(f"Got emb info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                print(f"FILE EXISTS: emb info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            print(f"ERROR: Unable to get session emb for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            unfinished_sessions.append((course, session_id))
            print(traceback.format_exc())
    

In [ ]:
sessionA_emb_info = pickle.load(open(f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info/{sessionA}-front.pb','rb'))
sessionB_emb_info = pickle.load(open(f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info/{sessionB}-front.pb','rb'))


In [ ]:
sessionA_id_map = pickle.load(open(f"{postprocessed_id_map_data_dir}/{sessionA}-front.pb","rb"))
sessionB_id_map = pickle.load(open(f"{postprocessed_id_map_data_dir}/{sessionB}-front.pb","rb"))


In [ ]:
sessionB_id_map[1295]


In [ ]:
sessionA_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{sessionA}/{sessionA}-front.avi'
sessionB_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{sessionB}/{sessionB}-front.avi'


## Replace raw ids with mapped ids for both sessions


In [ ]:
sessionA_emb_info = {
    xr:{
        sessionA_id_map[yr]:sessionA_emb_info[xr][yr] 
            for yr in sessionA_emb_info[xr] if not (sessionA_id_map[yr]==10000)} for xr in sessionA_emb_info}


In [ ]:
sessionB_emb_info = {
    xr:{
        sessionB_id_map[yr]:sessionB_emb_info[xr][yr] 
            for yr in sessionB_emb_info[xr] if not (sessionB_id_map[yr]==10000)} for xr in sessionB_emb_info}


In [ ]:
len(sessionA_emb_info.keys()), len(sessionB_emb_info.keys())


In [ ]:
# sessionA_emb_info[0][1]['face_embedding'].tolist()


In [ ]:
# arrange info as per tracking id across both sessions
# pitch, roll, yaw= sessionA_emb_info[0][9]['rvec'][0]
gaze_infoA = {}
emb_infoA = {}
for frame_number in sessionA_emb_info:
    for trackId in sessionA_emb_info[frame_number]:
        if trackId not in gaze_infoA:
            gaze_infoA[trackId] = []
            emb_infoA[trackId]=[]
        # get  gaze info
        try:
            pitch, roll, yaw= sessionA_emb_info[frame_number][trackId]['rvec'][0]
            pitch, roll, yaw=np.rad2deg(pitch), np.rad2deg(roll), np.rad2deg(yaw)
            gaze_sx, gaze_sy, gaze_ex, gaze_ey = sessionA_emb_info[frame_number][trackId]['gaze_2d'][0].flatten()
            gaze_infoA[trackId].append([frame_number, pitch, roll, yaw, gaze_sx, gaze_sy, gaze_ex, gaze_ey])
            face_emb = sessionA_emb_info[frame_number][trackId]['face_embedding'].tolist()
            emb_infoA[trackId].append([frame_number]+face_emb)
        except:
            continue

for id in gaze_infoA:
    gaze_infoA[id] = pd.DataFrame(gaze_infoA[id], columns=['frame','pitch','roll','yaw','gaze_sx', 'gaze_sy', 'gaze_ex', 'gaze_ey']).set_index('frame')
    emb_infoA[id] =pd.DataFrame(emb_infoA[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')

gaze_infoB = {}
emb_infoB = {}
for frame_number in sessionB_emb_info:
    for trackId in sessionB_emb_info[frame_number]:
        if trackId not in gaze_infoB:
            gaze_infoB[trackId] = []
            emb_infoB[trackId]=[]
        # get  gaze info
        try:
            pitch, roll, yaw= sessionB_emb_info[frame_number][trackId]['rvec'][0]    
            gaze_sx, gaze_sy, gaze_ex, gaze_ey = sessionB_emb_info[frame_number][trackId]['gaze_2d'][0].flatten()
            pitch, roll, yaw=np.rad2deg(pitch), np.rad2deg(roll), np.rad2deg(yaw)
            gaze_infoB[trackId].append([frame_number, pitch, roll, yaw, gaze_sx, gaze_sy, gaze_ex, gaze_ey])
            face_emb = sessionB_emb_info[frame_number][trackId]['face_embedding'].tolist()
            emb_infoB[trackId].append([frame_number]+face_emb)
        except:
            continue


for id in gaze_infoB:
    gaze_infoB[id] = pd.DataFrame(gaze_infoB[id], columns=['frame','pitch','roll','yaw','gaze_sx', 'gaze_sy', 'gaze_ex', 'gaze_ey']).set_index('frame')
    emb_infoB[id] =pd.DataFrame(emb_infoB[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')

len(gaze_infoA.keys()), len(gaze_infoB.keys())


In [ ]:
sprint({xr:(gaze_infoA[xr].shape[0],emb_infoA[xr].shape[0]) for xr in emb_infoA})


In [ ]:
sprint({xr:(gaze_infoB[xr].shape[0],emb_infoB[xr].shape[0]) for xr in emb_infoB})


In [ ]:
sprint({xr:(gaze_infoA[xr].shape[0],emb_infoA[xr].shape[0]) for xr in emb_infoA})


In [ ]:
sprint({xr:(gaze_infoB[xr].shape[0],emb_infoB[xr].shape[0]) for xr in emb_infoB})


### Finer debugging


In [ ]:
gaze_infoA[2].head(30)


In [ ]:
frame_number, frame_data = pickle.load(open(f'{sessionA_frame_dir}/7326.pb','rb'))
frame_data = {sessionA_id_map[int(xr['track_id'])]:xr for xr in frame_data if not (int(xr['track_id'])==10000)}
frame_data[10]


In [ ]:
import mmcv
mmcv_video_frames = mmcv.VideoReader(sessionA_video_file)


In [ ]:

for frame_number, video_frame in enumerate(mmcv_video_frames):
    print(frame_number)
    if frame_number==174:
        break


In [ ]:
plt.imshow(video_frame)


In [ ]:
sessionA_id_map[4]


In [ ]:
frame_number, list(map(int, list(frame_data[0]['face'][0])))


In [ ]:
# sprint({xr:emb_infoB[xr].shape for xr in emb_infoB})
from facenet_pytorch import InceptionResnetV1
facial_embedding_model = InceptionResnetV1(pretrained='vggface2',
                                           device='cpu').eval()


In [ ]:
frame_result = frame_data[2]
body_bbox = frame_result['bbox']
faces = frame_result.get('face',np.array([]))
X_TL, Y_TL, X_BR, Y_BR = body_bbox[:4].astype(int)
X_TL, Y_TL, X_BR, Y_BR


In [ ]:
print(faces[0])
faces[0][0] += X_TL
faces[0][1] += Y_TL
faces[0][2] += X_TL
faces[0][3] += Y_TL


In [ ]:
faces = faces[0][:4].astype(int)


In [ ]:
faces


# Ground truth for id matching


In [ ]:
gt_str = '''
3,12
3,36
11,17
30,17
5,10
5,35
9,9
1,0
6,8
4,8
14,1
16,6
21,4
7,2
7,35
0,5
-1,20
13,3
32,7
24,7
26,7
10,7
19,14
19,11
15,-1
2,-1
8,-1
-1,16
-1,26
-1,18
'''


In [ ]:
gt_arr = gt_str.split("\n")[1:-1]
gt_arr = [xr.split(",") for xr in gt_arr]
df_gt = pd.DataFrame(gt_arr, columns=[sessionA,sessionB])
gt_map = df_gt.groupby(sessionA).agg({sessionB:lambda x: list(x)})[sessionB].to_dict()
gt_map


# Method 1: Direct match between ids post filtering and median


In [ ]:
idA = 9
idB = 4


In [ ]:
framesA = gaze_infoA[idA][(gaze_infoA[idA].yaw.abs()<10) & (gaze_infoA[idA].pitch.abs()<20) & (gaze_infoA[idA].roll.abs()<20)].index.values
framesB = gaze_infoB[idB][(gaze_infoB[idB].yaw.abs()<10) & (gaze_infoB[idB].pitch.abs()<20) & (gaze_infoB[idB].roll.abs()<20)].index.values
len(framesA), len(framesB)


In [ ]:
framesA_emb = emb_infoA[idA].loc[framesA]
framesB_emb = emb_infoB[idB].loc[framesB]
framesA_emb.shape, framesB_emb.shape


In [ ]:
medianA_emb = np.median(emb_infoA[idA].loc[framesA],axis=0)
medianB_emb = np.median(emb_infoB[idB].loc[framesB],axis=0)
sprint(medianA_emb.shape, medianB_emb.shape)
sprint(cdist(medianA_emb.reshape(1,-1), medianB_emb.reshape(1,-1)))


In [ ]:
dist_mat = cdist(framesA_emb.values, framesB_emb.values)
dist_mat.shape, np.median(dist_mat)


In [ ]:
sns.histplot(np.mean(dist_mat,axis=1),bins=100)


In [ ]:
self_dist_matA = cdist(framesA_emb.values, framesA_emb.values,'cosine')
sns.histplot(np.mean(self_dist_matA,axis=1),bins=100)


In [ ]:
self_dist_matB = cdist(framesB_emb.values, framesB_emb.values,'cosine')
sns.histplot(np.mean(self_dist_matB,axis=1),bins=100)


In [ ]:
# plot median embedding for A and B
plt.figure(figsize=(40,10))
plt.plot(medianA_emb)
plt.plot(medianB_emb)


## First experiment (to be used for method 3)


In [ ]:
gaze_infoB.keys()


In [ ]:
gaze_infoA[0].head()


In [ ]:
emb_infoA[0].head()


In [ ]:
np.nanmax([gaze_infoA[xr].roll.max() for xr in gaze_infoA])


In [ ]:
angle_bins = np.arange(-150,330,60)
angle_bins


In [ ]:
for id in gaze_infoA:
    gaze_infoA[id]['roll_cat'] = [np.argmax(angle_bins>xr) for xr in gaze_infoA[id].roll]
    gaze_infoA[id]['pitch_cat'] = [np.argmax(angle_bins>xr) for xr in gaze_infoA[id].pitch]
    gaze_infoA[id]['yaw_cat'] = [np.argmax(angle_bins>xr) for xr in gaze_infoA[id].yaw]

for id in gaze_infoB:
    gaze_infoB[id]['roll_cat'] = [np.argmax(angle_bins>xr) for xr in gaze_infoB[id].roll]
    gaze_infoB[id]['pitch_cat'] = [np.argmax(angle_bins>xr) for xr in gaze_infoB[id].pitch]
    gaze_infoB[id]['yaw_cat'] = [np.argmax(angle_bins>xr) for xr in gaze_infoB[id].yaw]
    

In [ ]:
# Data is ready, Now match id in one session to another session


In [ ]:
df_temp = gaze_infoA[1].reset_index().groupby(['roll_cat', 'pitch_cat', 'yaw_cat'],as_index=False).agg({'frame':[lambda x: list(x), 'count']})
df_temp.columns = ['roll_cat', 'pitch_cat', 'yaw_cat','list','count']
df_temp = df_temp.sort_values(by='count', ascending=False)
df_temp


In [ ]:
temp_gaze_framesA = df_temp.iloc[1]['list']
temp_roll, temp_pitch,temp_yaw= df_temp.iloc[1][['roll_cat','pitch_cat','yaw_cat']].values.tolist()
# temp_gaze_framesA
temp_roll, temp_pitch,temp_yaw


In [ ]:
tmp_emb = emb_infoA[1][emb_infoA[1].index.isin(temp_gaze_framesA)]
tmp_emb


In [ ]:
from sklearn.metrics import pairwise_distances


In [ ]:
dist_mat = pairwise_distances(tmp_emb.values[:,:])
# sns.heatmap(dist_mat)


In [ ]:
dist_thr = np.median(np.median(dist_mat, axis=1))
allowed_idx = np.median(dist_mat, axis=1)<dist_thr
dist_thr


In [ ]:
dist_mat = dist_mat[allowed_idx, :]
dist_mat = dist_mat[:,allowed_idx]
dist_mat.shape


In [ ]:
np.median(dist_mat, axis=1)<dist_thr
dist_mat.shape


In [ ]:
sns.heatmap(dist_mat)


In [ ]:
gaze_infoA.keys()


In [ ]:
tmp_emb.values


In [ ]:
dist_mat.shape


In [ ]:
# get embedding matrix for all gaze categories
@timer
def get_emb_matrices(df_id_gaze, df_id_emb, max_dist_threshold=0.5):
    df_gaze_cats = df_id_gaze.reset_index().groupby(['roll_cat', 'pitch_cat', 'yaw_cat'],as_index=False).agg({'frame':[lambda x: list(x), 'count']})
    df_gaze_cats.columns = ['roll_cat', 'pitch_cat', 'yaw_cat','list','count']
    df_gaze_cats = df_gaze_cats.sort_values(by='count', ascending=False)
    emb_matrices = {}
    for row_idx, row in df_gaze_cats.iterrows():
        temp_gaze_framesA = row['list']
        temp_roll, temp_pitch,temp_yaw= row['roll_cat'], row['pitch_cat'], row['yaw_cat']
    
        # get embedding matrix
        tmp_emb = df_id_emb[df_id_emb.index.isin(temp_gaze_framesA)]
    
        #filter outlier embeddings
        dist_mat = pairwise_distances(tmp_emb.values[:,:])
        dist_thr = min(np.median(np.median(dist_mat, axis=1)), max_dist_threshold)
        allowed_idx = np.median(dist_mat, axis=1)<dist_thr
    
        emb_matrix = tmp_emb.iloc[allowed_idx,:]
        emb_matrices[(temp_roll, temp_pitch,temp_yaw)] = emb_matrix
    return emb_matrices
    

emb_matsA = {}
for id in gaze_infoA.keys():
    sprint(id)
    emb_matsA[id] = get_emb_matrices(gaze_infoA[id],emb_infoA[id])
    # break


In [ ]:

emb_matsB = {}
for id in gaze_infoB.keys():
    sprint(id)
    emb_matsB[id] = get_emb_matrices(gaze_infoB[id],emb_infoB[id])


In [ ]:
# matching score between different ids
temp_s1_mats=emb_matsA[0]
temp_s2_mats=emb_matsB[0]


In [ ]:
common_keys = [xr for xr in temp_s1_mats.keys() if xr in temp_s2_mats.keys()]
common_keys


In [ ]:
s1_emb_mat_key = temp_s1_mats[common_keys[0]]
s2_emb_mat_key = temp_s2_mats[common_keys[0]]
s1_emb_mat_key.shape, s2_emb_mat_key.shape


In [ ]:
from scipy.spatial.distance import cdist
# dist_res = cdist(s1_emb_mat_key, s2_emb_mat_key)


In [ ]:
dist_res.shape


In [ ]:
# sns.heatmap(dist_res)


In [ ]:
match_score = np.median(dist_res)


In [ ]:
emb_matsA.keys()


In [ ]:
# get match scores between one single id from sessionA, and across all ids from sessionB
temp_idA_scores = dict()
temp_s1_mats=emb_matsA[697]
for idB in emb_matsB:
    temp_s2_mats = emb_matsB[idB]
    common_keys = [xr for xr in temp_s1_mats.keys() if xr in temp_s2_mats.keys()]
    temp_idA_scores[idB]={}
    for gaze_key in common_keys:
        s1_emb_mat_key = temp_s1_mats[gaze_key]
        s2_emb_mat_key = temp_s2_mats[gaze_key]
        dist_res = cdist(s1_emb_mat_key, s2_emb_mat_key)
        temp_idA_scores[idB][gaze_key] = np.median(dist_res)        
    


In [ ]:
df_idA = pd.DataFrame(temp_idA_scores)


In [ ]:
df_idA.min().sort_values()


In [ ]:
df_idA.min().sort_values()


In [ ]:
# get final id-match matrix between both sessions


In [ ]:
id_match_matrix = {}
for idA in emb_matsA:
    temp_s1_mats=emb_matsA[idA]
    id_match_matrix[idA]={}
    for idB in emb_matsB:
        temp_s2_mats = emb_matsB[idB]
        common_keys = [xr for xr in temp_s1_mats.keys() if xr in temp_s2_mats.keys()]
        # temp_idA_scores[idB]={}
        gaze_cat_scores= []
        for gaze_key in common_keys:
            s1_emb_mat_key = temp_s1_mats[gaze_key]
            s2_emb_mat_key = temp_s2_mats[gaze_key]
            dist_res = np.median(cdist(s1_emb_mat_key, s2_emb_mat_key))
            if dist_res >= 0:
                gaze_cat_scores.append(dist_res)
            # temp_idA_scores[idB][gaze_key] = np.median(dist_res) 
        if len(gaze_cat_scores)>0:
            id_match_matrix[idA][idB] = np.nanmin(gaze_cat_scores)


In [ ]:
df_id_match = pd.DataFrame(id_match_matrix)


In [ ]:
plt.figure(figsize=(30,10))
sns.heatmap(df_id_match.round(2), annot=True, cmap='bone_r')


In [ ]:
df_id_match


# Clean Implementation 1: Using k% gaze filtering and median embeddings for matching


In [ ]:
MAX_GAZE_DEVIATION_DEG = 30
# MAX_EMBEDDING_FRAMES = 1000
MIN_EMBEDDING_FRAMES = 100


In [ ]:
match_scores = {}
match_info = {}
np.random.seed(42)
for idA,idB in product(emb_infoA.keys(), emb_infoB.keys()):
    #filter correct frames
    framesA = gaze_infoA[idA][
        (gaze_infoA[idA].yaw.abs()<MAX_GAZE_DEVIATION_DEG) & 
        (gaze_infoA[idA].pitch.abs()<MAX_GAZE_DEVIATION_DEG) & 
        (gaze_infoA[idA].roll.abs()<MAX_GAZE_DEVIATION_DEG)].index.values
    framesB = gaze_infoB[idB][
        (gaze_infoB[idB].yaw.abs()<MAX_GAZE_DEVIATION_DEG) &
        (gaze_infoB[idB].pitch.abs()<MAX_GAZE_DEVIATION_DEG) &
        (gaze_infoB[idB].roll.abs()<MAX_GAZE_DEVIATION_DEG)].index.values

    num_framesA, num_framesB = len(framesA), len(framesB)
    if (num_framesA<MIN_EMBEDDING_FRAMES) | (num_framesB <MIN_EMBEDDING_FRAMES):
        sprint(f"Not sufficient frames to match {idA}:{len(framesA)},{idB}:{len(framesB)}")
        continue
    
    # select_framesA  =  np.random.choice(framesA, MAX_EMBEDDING_FRAMES) if (num_framesA > MAX_EMBEDDING_FRAMES) else  framesA
    # select_framesB  =  np.random.choice(framesB, MAX_EMBEDDING_FRAMES) if (num_framesB > MAX_EMBEDDING_FRAMES) else  framesB

    #get id embeddings
    
    medianA_emb = np.median(emb_infoA[idA].loc[framesA],axis=0)
    medianB_emb = np.median(emb_infoB[idB].loc[framesB],axis=0)
    match_distance = cdist(medianA_emb.reshape(1,-1), medianB_emb.reshape(1,-1))[0][0]
    if idA not in match_scores:
        match_scores[idA] = {}
        match_info[idA] = {}
    match_scores[idA][idB] = match_distance
    match_info[idA][idB] = dict(match_score=match_distance, framesA=framesA, framesB=framesB)
    sprint(idA, idB, match_distance)
    
    
    
    

In [ ]:
df_matching_methodA = pd.DataFrame(match_scores)
df_matching_methodA.shape


In [ ]:
df_matching_methodA


In [ ]:
from matplotlib.patches import Rectangle
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching_methodA.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching_methodA.columns) and (gtB in df_matching_methodA.index):
                locA, locB = df_matching_methodA.columns.get_loc(gtA), df_matching_methodA.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching_methodA.shape[1]):
    locBs = df_matching_methodA.iloc[:,locA].argsort()[:3]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))


axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method1_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')


# Method 2:Get distance between all pairs and get median distance across all pairs.


In [ ]:
MAX_GAZE_DEVIATION_DEG = 30 # for basic filtering of very random gaze
MAX_EMBEDDING_FRAMES = 1000
MIN_EMBEDDING_FRAMES = 100


In [ ]:
match_scores = {}
matching_info_dictB = {}
np.random.seed(42)
for idA,idB in product(emb_infoA.keys(), emb_infoB.keys()):
    #filter correct frames
    framesA = gaze_infoA[idA][
        (gaze_infoA[idA].yaw.abs()<MAX_GAZE_DEVIATION_DEG) & 
        (gaze_infoA[idA].pitch.abs()<MAX_GAZE_DEVIATION_DEG) & 
        (gaze_infoA[idA].roll.abs()<MAX_GAZE_DEVIATION_DEG)].index.values
    framesB = gaze_infoB[idB][
        (gaze_infoB[idB].yaw.abs()<MAX_GAZE_DEVIATION_DEG) &
        (gaze_infoB[idB].pitch.abs()<MAX_GAZE_DEVIATION_DEG) &
        (gaze_infoB[idB].roll.abs()<MAX_GAZE_DEVIATION_DEG)].index.values

    num_framesA, num_framesB = len(framesA), len(framesB)
    if (num_framesA<MIN_EMBEDDING_FRAMES) | (num_framesB <MIN_EMBEDDING_FRAMES):
        sprint(f"Not sufficient frames to match {idA}:{len(framesA)},{idB}:{len(framesB)}")
        continue
    
    select_framesA  =  np.random.choice(framesA, MAX_EMBEDDING_FRAMES) if (num_framesA > MAX_EMBEDDING_FRAMES) else  framesA
    select_framesB  =  np.random.choice(framesB, MAX_EMBEDDING_FRAMES) if (num_framesB > MAX_EMBEDDING_FRAMES) else  framesB

    #get id embeddings
    
    match_distance_matrix = cdist(emb_infoA[idA].loc[select_framesA], emb_infoB[idB].loc[select_framesB])
    sprint(match_distance_matrix.shape)
    match_distance = np.mean(np.median(match_distance_matrix,axis=1))
    if idA not in match_scores:
        match_scores[idA] = {}
        matching_info_dictB[idA] = {}
    match_scores[idA][idB] = match_distance
    matching_info_dictB[idA][idB] = dict(match_score=match_distance, match_matrix= match_distance_matrix,framesA=select_framesA, framesB=select_framesB)
    sprint(idA, idB, match_distance)
    

df_matching_methodB = pd.DataFrame(match_scores)
    

In [ ]:
from matplotlib.patches import Rectangle
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching_methodB.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching_methodB.columns) and (gtB in df_matching_methodB.index):
                locA, locB = df_matching_methodB.columns.get_loc(gtA), df_matching_methodB.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching_methodB.shape[1]):
    locBs = df_matching_methodB.iloc[:,locA].argsort()[:3]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))


axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method2_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')


## Method 2b: Use median of medians


In [ ]:
match_scores = {}
for idA in matching_info_dictB:
    match_scores[idA] = {}
    for idB in matching_info_dictB[idA]:
        match_matrix = matching_info_dictB[idA][idB]['match_matrix']
        match_distance = np.median(np.median(match_matrix,axis=1))
        match_scores[idA][idB] = match_distance

df_matching = pd.DataFrame(match_scores)

#--------
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching.columns) and (gtB in df_matching.index):
                locA, locB = df_matching.columns.get_loc(gtA), df_matching.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching.shape[1]):
    locBs = df_matching.iloc[:,locA].argsort()[:3]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))


axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method2b_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')


In [ ]:
match_scores = {}
for idA in matching_info_dictB:
    match_scores[idA] = {}
    for idB in matching_info_dictB[idA]:
        match_matrix = matching_info_dictB[idA][idB]['match_matrix']
        match_distance = np.median(np.min(match_matrix,axis=1))
        match_scores[idA][idB] = match_distance

df_matching = pd.DataFrame(match_scores)

#--------
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching.columns) and (gtB in df_matching.index):
                locA, locB = df_matching.columns.get_loc(gtA), df_matching.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching.shape[1]):
    locBs = df_matching.iloc[:,locA].argsort()[:3]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))


axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method2c_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')


In [ ]:
from sklearn.preprocessing import normalize
match_scores = {}
for idA in matching_info_dictB:
    match_scores[idA] = {}
    for idB in matching_info_dictB[idA]:
        match_matrix = deepcopy(matching_info_dictB[idA][idB]['match_matrix'])
        # match_matrix = normalize(match_matrix, axis=1, norm='l1')
        match_distance = np.median(match_matrix.flatten())
        match_scores[idA][idB] = match_distance

df_matching = pd.DataFrame(match_scores)

#--------
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching.columns) and (gtB in df_matching.index):
                locA, locB = df_matching.columns.get_loc(gtA), df_matching.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching.shape[1]):
    locBs = df_matching.iloc[:,locA].argsort()[:3]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))


axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method2d_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')


# Method 3: Use spectral clustering to get clean set of embeddings, calculate their centroid, and then evaluate distance


In [ ]:
from sklearn.cluster import DBSCAN
EPS = 0.4
MIN_PTS = 100


In [ ]:

np.random.seed(42)
clustered_median_embA = {}
for idA in emb_infoA:
    emb_cluA = DBSCAN(min_samples=MIN_PTS, eps=EPS)
    emb_cluA.fit(emb_infoA[idA].values)
    if max(emb_cluA.labels_)<0:
        sprint(f"All frames are outliers, not proceeding with id {idA}")
        continue
    best_cluster_id = pd.Series(emb_cluA.labels_[emb_cluA.labels_>=0]).value_counts().index[0]
    framesA = emb_infoA[idA].iloc[emb_cluA.labels_==best_cluster_id].index.values
    clustered_median_embA[idA] = np.median(emb_infoA[idA].loc[framesA],axis=0)
    
clustered_median_embB = {}
for idB in emb_infoB:
    emb_cluB = DBSCAN(min_samples=MIN_PTS, eps=EPS)
    emb_cluB.fit(emb_infoB[idB].values)
    if max(emb_cluB.labels_)<0:
        sprint(f"All frames are outliers, not proceeding with id {idB}")
        continue
    best_cluster_id = pd.Series(emb_cluB.labels_[emb_cluB.labels_>=0]).value_counts().index[0]
    framesB = emb_infoB[idB].iloc[emb_cluB.labels_==best_cluster_id].index.values
    clustered_median_embB[idB] = np.median(emb_infoB[idB].loc[framesB],axis=0)
# #filter correct frames
# # select_framesA  =  np.random.choice(framesA, MAX_EMBEDDING_FRAMES) if (num_framesA > MAX_EMBEDDING_FRAMES) else  framesA
# # select_framesB  =  np.random.choice(framesB, MAX_EMBEDDING_FRAMES) if (num_framesB > MAX_EMBEDDING_FRAMES) else  framesB

# #get id embeddings
    
# medianA_emb = np.median(emb_infoA[idA].loc[framesA],axis=0)
# medianB_emb = np.median(emb_infoB[idB].loc[framesB],axis=0)
# match_distance = cdist(medianA_emb.reshape(1,-1), medianB_emb.reshape(1,-1))[0][0]
# if idA not in match_scores:
#     match_scores[idA] = {}
#     match_info3[idA] = {}
# match_scores[idA][idB] = match_distance
# match_info3[idA][idB] = dict(match_score=match_distance, framesA=framesA, framesB=framesB)
# sprint(idA, idB, match_distance)

    
    
    

In [ ]:
match_scores = {}
match_info3 = {}
for idA in clustered_median_embA:
    match_scores[idA] = {}
    for idB in clustered_median_embB:
        match_scores[idA][idB] = cdist(clustered_median_embA[idA].reshape(1,-1), clustered_median_embB[idB].reshape(1,-1))[0][0]
        
df_matching_methodC = pd.DataFrame(match_scores)
df_matching_methodC.shape    


In [ ]:
from matplotlib.patches import Rectangle
fig, axn = plt.subplots(1,1,figsize=(20,10))
sns.heatmap(df_matching_methodC.round(2), annot=True,ax=axn,cmap='bone_r')
for gtA in map(int, gt_map):
    for gtB in map(int, gt_map[str(gtA)]):
        if (gtA>=0) and (gtB>=0):
            if (gtA in df_matching_methodC.columns) and (gtB in df_matching_methodC.index):
                locA, locB = df_matching_methodC.columns.get_loc(gtA), df_matching_methodC.index.get_loc(gtB)
                axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='red', lw=4))

for locA in range(df_matching_methodC.shape[1]):
    locBs = df_matching_methodC.iloc[:,locA].argsort()[:3]
    for locB in locBs:
        axn.add_patch(Rectangle((locA, locB), 1, 1, fill=False, edgecolor='blue', lw=1))


axn.set_xlabel(f"Session A: {sessionA}",fontsize=16)
axn.set_ylabel(f"Session B: {sessionB}",fontsize=16)
plt.savefig(f'plots/Method3_{course}_{sessionA.split("_")[-1]}_{sessionB.split("_")[-1]}.png',dpi=400,bbox_inches='tight')
